In [18]:
import sympy as sp
import pickle

In [19]:
# Define dynamical variables and system parameters
symbols = sp.symbols("theta1 theta2 theta_1^' theta_2^' \\theta_1^{''} \\theta_2^{''} m1 m2 l1 l2 g")
th1, th2, th1d, th2d, th1dd, th2dd, m1, m2, l1, l2, g = symbols

# Take the time derivative of a function of variables that can depend on time
def ddt(f):
    result = 0
    d = {th1: th1d, th2: th2d, th1d: th1dd, th2d: th2dd}
    for s in f.free_symbols.intersection(set(d)):
        result += sp.diff(f, s) * d[s]
    return result

# Codify Cartesian to generalized coord transform
x1 = l1*sp.sin(th1)
x2 = x1 + l2*sp.sin(th2)
y1 = -l1*sp.cos(th1)
y2 = y1 - l2*sp.cos(th2)

# Compute Lagrangian
x1d = ddt(x1)
x2d = ddt(x2)
y1d = ddt(y1)
y2d = ddt(y2)
T = 1/2*(m1*(x1d**2 + y1d**2) + m2*(x2d**2 + y2d**2))
U = m1*g*y1 + m2*g*y2
L = T - U

# Compute Euler-Lagrange equations
lhs1 = sp.diff(L, th1)
rhs1 = ddt(sp.diff(L, th1d))
lhs2 = sp.diff(L, th2)
rhs2 = ddt(sp.diff(L, th2d))

In [20]:
# Substitute/rearrange to get equations in proper form for solve_ivp
sol1 = sp.solve(lhs1 - rhs1, th1dd)[0]
sol2 = sp.solve(lhs2 - rhs2, th2dd)[0]
theta1dd = sp.solve(th1dd - sol1.subs(th2dd, sol2), th1dd)[0]
theta2dd = sp.solve(th2dd - sol2.subs(th1dd, sol1), th2dd)[0]
display(sp.Eq(th1dd, theta1dd))
display(sp.Eq(th2dd, theta2dd))

Eq(\theta_1^{''}, (-g*m1*sin(theta1) - g*m2*sin(theta1)/2 - g*m2*sin(theta1 - 2*theta2)/2 - l1*m2*theta_1^'**2*sin(2*theta1 - 2*theta2)/2 - l2*m2*theta_2^'**2*sin(theta1 - theta2))/(l1*(m1 - m2*cos(theta1 - theta2)**2 + m2)))

Eq(\theta_2^{''}, (-g*m1*sin(theta2)/2 + g*m1*sin(2*theta1 - theta2)/2 - g*m2*sin(theta2)/2 + g*m2*sin(2*theta1 - theta2)/2 + l1*m1*theta_1^'**2*sin(theta1 - theta2) + l1*m2*theta_1^'**2*sin(theta1 - theta2) + l2*m2*theta_2^'**2*sin(2*theta1 - 2*theta2)/2)/(l2*(m1 - m2*cos(theta1 - theta2)**2 + m2)))

In [5]:
with open("symbols.pickle", "wb") as f:
    pickle.dump((th1, th2, th1d, th2d, m1, m2, l1, l2, g), f, pickle.HIGHEST_PROTOCOL)
with open("equations.pickle", "wb") as f:
    pickle.dump((theta1dd, theta2dd), f, pickle.HIGHEST_PROTOCOL)